In [1]:
import os 
import requests
from openai import OpenAI

In [2]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
key = "put-key"
client = OpenAI (api_key=key)

In [127]:
def upload_file(file:str):
    """Takes a file path as an argument and sends a POST request to the OpenAI API."""
    # Check if the file exists
    if not os.path.isfile(file):
        print(f"Error: The file '{file}' does not exist.")
        return None
    url = "https://api.openai.com/v1/files"
    headers = { 'Authorization': f'Bearer {key}' }
    data = { "purpose": "fine-tune" }
    # Use 'with' to ensure the file is properly closed after use
    try:
        with open(file, "rb") as f:
            files = { "file": f }
            response = requests.post(url, headers=headers, data=data, files=files, timeout=20)
        # Check for response status
        if response.status_code == 200:
            result = response.json()
            return result
        else:
            print(f"Request failed with status code: {response.status_code}")
            print("Error response:", response.text)
    except requests.exceptions.RequestException as e:
        # Catch any network-related errors
        print(f"An error occurred while making the request: {e}")
    
    return None

In [128]:
def fine_tuning_model(file_id: str):
    """Fine-tunes a model using a JSONL file."""
    
    url = "https://api.openai.com/v1/fine_tuning/jobs"
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {key}'
    }
    data = { 
        "training_file": file_id, 
        "model": "gpt-3.5-turbo" 
    }
    
    try:
        response = requests.post(url, headers=headers, json=data, timeout=20)

        # Handle success
        if response.status_code == 200:
            result = response.json()
            return result
        else:
            # Handle non-success status codes with more details
            print(f"Request failed with status code: {response.status_code}")
            print("Response text:", response.text)
    
    except requests.exceptions.RequestException as e:
        # Catch any network or request-related errors
        print(f"An error occurred while making the request: {e}")
    
    return None

In [129]:
def retrieve_fine_tuning(fine_tune_id):
    """Retrieves a Fine Tune Model"""
    url = f"https://api.openai.com/v1/fine_tuning/jobs/{fine_tune_id}"
    headers = {
    'Authorization': f'Bearer {key}'
    }
    response = requests.get(url, headers=headers, timeout=20)
    print(response.text)
    if response.status_code == 200:
        result = response.json()
        return result
    print ('Request failed with status code:', response.status_code) 
    return None

In [130]:
upload_result = upload_file("./data.jsonl")
uploaded_file_id = upload_result["id"]

In [131]:
fine_tune_model_result = fine_tuning_model(uploaded_file_id)
fine_tune_model_id = fine_tune_model_result["id"]

In [ ]:
fine_tuned_model = ""
while True:
    fine_tune_object = retrieve_fine_tuning(fine_tune_model_id)
    if fine_tune_object["status"] == "succeeded":
        fine_tuned_model = fine_tune_object
        break

{
  "object": "fine_tuning.job",
  "id": "ftjob-YRVdBYFik4ClM53nyQPbtybh",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1728002914,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-HDXkwU0SyxJNfKQl7W7C5OEm",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-jr0L1SEbrSdQLKWSv3yCaePR",
  "hyperparameters": {
    "n_epochs": "auto",
    "batch_size": "auto",
    "learning_rate_multiplier": "auto"
  },
  "trained_tokens": null,
  "error": {},
  "user_provided_suffix": null,
  "seed": 14873992,
  "estimated_finish": null,
  "integrations": []
}
{
  "object": "fine_tuning.job",
  "id": "ftjob-YRVdBYFik4ClM53nyQPbtybh",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1728002914,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-HDXkwU0SyxJNfKQl7W7C5OEm",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-jr0L1SEbrSdQLKWSv3yC

In [ ]:
print(fine_tuned_model['fine_tuned_model'])

In [123]:
completion_response = client.chat.completions.create(
    model=fine_tuned_model['fine_tuned_model'], messages=[
    {"role": "system", "content": "Client is a user searching a house"}, 
        {"role": "user", "content": "I need a home in california nice view?"}]
    )
print (completion_response)

ChatCompletion(id='chatcmpl-AEQc3vfG9XEtdxTCFgfjmV78PtCm6', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Sure, I can help you with that. Could you please provide me with more details about your preferences for the home, such as the number of bedrooms and bathrooms, preferred location in California, and any other specific features you are looking for in a home with a nice view?', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1728002255, model='ft:gpt-3.5-turbo-0125:personal::AEQWhbiH', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=55, prompt_tokens=27, total_tokens=82, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)))
